In [126]:
import requests
import json

# USER: “I am interested in investing more on companies active on climate change. What are some of the news about that?”
# ** vector search on RSS news **
# BOT: “Here some relevant news about..”
# USER: “Could you suggest a fund with a focus on this?”
# ** vector serach on funds **
# BOT: “You might be interested in the CCSF found”
# USER: “Cool. What are some of the risks associated with this investment?”
# ** vector search on funds_chunks **
# BOT: “Some of the risks are blabla”

def firstKeyVal(d):
    return list(d.values())[0]

def formatResult(result):
    metadata = "\n".join(result.get('tags',[]) + result.get('nasdaq_tickers',[]))
    title = firstKeyVal(result['title'])
    summary = firstKeyVal(result['summary'])
    content = "\n".join(firstKeyVal(result['content']))

    return f"# Title\n{title}\n# Summary\n{summary}\n# Content\n{content}\n# Metadata\n{metadata}\n"

def runChat(queryPrompt,systemPrompt):
    return requests.post("https://eu-central-1.aws.data.mongodb-api.com/app/shif-ibfjf/endpoint/getGpt35turbo",json={"prompt": queryPrompt,"systemPrompt": systemPrompt}).json()

def runChat4(queryPrompt,systemPrompt):
    return requests.post("https://eu-central-1.aws.data.mongodb-api.com/app/shif-ibfjf/endpoint/getGpt4",json={"prompt": queryPrompt,"systemPrompt": systemPrompt}).json()

def runResultPicker(query,formattedResults):
    systemPrompt = """You are a financial analyst providing advice and guidance to a client"""
    queryPrompt = f"""Pick the 5 results that can best be used to address the client's query: {query}.
    Explain why you chose them.
    You must provide your answer in the form:{{"results":[<list of {{"title":<result title>,"content":<result content>}}>],"explanation":<explanation>}}
    Here are the results:
    {formattedResults}"""
    return runChat(queryPrompt,systemPrompt)

def formatResults(results):
    return "\n".join([f"{i+1}. From: {result.get('parent_link','')}\n{result['content']}" for i, result in enumerate(results)])

def vectorSearch(query,collName,searchIndex,fields):
    r = requests.post(f"https://eu-central-1.aws.data.mongodb-api.com/app/shif-ibfjf/endpoint/vectorSearch?query={query}&limit=10&numCandidates=100&collName={collName}&searchIndex={searchIndex}&fields={fields}").json()
    results = r['results']
    metadata = r['metadata']
    return {
        "results": results,
        "metadata": metadata,
        "formatted": formatResults(results)
    }

def textSearch(query,collName,searchIndex,fields):
    r = requests.get(f"https://eu-central-1.aws.data.mongodb-api.com/app/shif-ibfjf/endpoint/textSearch?query={query}&limit=10&collName={collName}&searchIndex={searchIndex}&fields={fields}").json()
    results = r['results']
    metadata = r['metadata']
    return {
        'results': results,
        'metadata': metadata,
        "formatted": formatResults(results)
    }

In [128]:
query = "I am interested in investing more on companies active on climate change."
systemPrompt = """You are a reasoning engine tasked with providing financial guidance.
You have access to the following data in MongoDB:
- A collection of nasdaq and business news article chunks with the following fields: title, summary, content, tags, nasdaq_tickers.
- A collection of fund chunks with the following fields: fund_name, content.
- A lexical search index on the chunked data which is good for matching keywords and phrases.
- A vector search index on the embedded chunked data which is good for semantic search.
"""
queryPrompt = f"""Create three phrases which can be embedded for vector search, and three phrases for lexical search, to retrieve data to use in a gpt prompt to answer this query: {query}
You must provide your answer in the form {{"vector":[<vector search phrases>], "lexical":[<lexical search phrases>]}}"""

searchPhrases = runChat(queryPrompt,systemPrompt)
print(searchPhrases)

{"vector":["climate change companies", "sustainable investing", "carbon footprint reduction"], "lexical":["climate change", "environmental sustainability", "renewable energy firms"]}


In [100]:
queries = json.loads(searchPhrases)
print(queries)
all_vector_results = [vectorSearch(query,'docs_chunks','vectorIndex','content,parent_link') for query in queries['vector']]
all_lexical_results = [textSearch(query,'docs_chunks','searchIndex','content,parent_link') for query in queries['lexical']]

{'vector': ['companies taking action on climate change', 'renewable energy industry leaders', 'green initiatives in business'], 'lexical': ['climate change', 'renewable energy', 'CO2 emissions reduction']}


In [121]:
formatted_vector_results = '\n'.join([r['formatted'] for r in all_vector_results])
formatted_lexical_results = '\n'.join([r['formatted'] for r in all_lexical_results])

vector_summary = json.loads(runResultPicker(query,formatted_vector_results))
lexical_summary = json.loads(runResultPicker(query,formatted_lexical_results))

print(vector_summary['explanation'],vector_summary['results'])
print(lexical_summary['explanation'],lexical_summary['results'])

I chose these results because they provide information about investing in companies that are taking action or involved in climate change, such as tracking global methane emissions, involvement in low-carbon fuel production, and exposure to companies leading the transition towards a low-carbon economy. These results also mention regulations related to methane pollution and reducing fashion's environmental impact. [{'title': 'Google-backed satellite to track global oil industry methane emissions', 'content': 'We\'ll be able to see who the laggards are, but hopefully they will use that information in a constructive way to improve their performance," said  Mark Brownstein, senior vice president for energy transition at EDF.'}, {'title': 'Oil Unchanged as Traders Take Profits on News of Expected Extension of Voluntary Output Cuts by OPEC+', 'content': "In a significant development for the advancement of low-carbon fuel production, Summit Carbon Solutions (Summit) announced that\xa0Valero, t

In [94]:

stocksSystemPrompt = """You are a financial analyst providing advice and guidance to a client"""
stocksQueryPrompt = f"""Review the following information with respect to the client's query: {query}.
You must provide a list of stock symbols that would be of interest in the form:{{"stocks":[<list of symbols>]}}
Here are the results of the vector searches:
{formatted_vector_results}
Here are the results of the lexical searches:
{formatted_lexical_results}
Only provide the stock symbols as a valid json response. Do not provide any other information."""

stocksAnswer = runChat(stocksQueryPrompt,stocksSystemPrompt)
print(json.loads(stocksAnswer))

{'stocks': ['MS', 'NFE']}


This cell is not working for some reason. Maybe context prompt is too long for the window?

In [79]:
# fundsVectorSearches = [vectorSearch(query,'funds_chunks','vectorIndex','content,nasdaq_tickers,tags,fund_name') for query in queries['vector']]
# fundsLexicalSearches = [textSearch(query,'funds_chunks','searchIndex','content,nasdaq_tickers,tags,fund_name') for query in queries['lexical']]

# formatted_funds_vector_results = '\n'.join([r['formatted'] for r in fundsVectorSearches])
# formatted_funds_lexical_results = '\n'.join([r['formatted'] for r in fundsLexicalSearches])

# fundsSummarySystemPrompt = """You are a financial analyst providing advice and guidance to a client"""
# fundsSummaryQueryPrompt = f"""Pick the ten results that can best be used to address the client's query: {query}.
# You must also pick out the most relevant funds for the client's query.
# You must provide your answer in the form:{{"funds":[<list of fund names>],"results":[{{"title":<result title>,"content":<result content>}}]}}
# Here are the results of the vector searches:
# {formatted_funds_vector_results}
# Here are the results of the lexical searches:
# {formatted_funds_lexical_results}"""


# fundsSummaryAnswer = runChat4(fundsSummaryQueryPrompt,fundsSummarySystemPrompt)
# print(json.loads(fundsSummaryAnswer))

In [124]:
stocks = set(json.loads(stocksAnswer)['stocks'])
links = set()
for r in all_vector_results+all_lexical_results:
    stocks.update(r['metadata']['stock_symbols'])
    links.update(r['metadata']['links'])

print(stocks)
print(links)
vector_formatted = "\n".join([f"# {r['title']}\n# Content\n{r['content']}" for r in vector_summary['results']])
lexical_formatted = "\n".join([f"# {r['title']}\n# Content\n{r['content']}" for r in lexical_summary['results']])
links_string = "\n".join(list(links)[:10])
stocks_string = ",".join(stocks)
finalSystemPrompt = """You are a financial analyst providing advice and guidance to a client."""
finalUserPrompt = f"""Provide a detailed summary of the information you have gathered for the client's query: {query}.
You must reference specific companies, funds and stocks and news articles where appropriate.
Vector Search results:
{vector_formatted}
Explanation:
{vector_summary['explanation']}
Lexical Search results:
{lexical_formatted}
Explanation:
{lexical_summary['explanation']}
relevant links:
{links_string}"""

finalAnswer = runChat(finalUserPrompt,finalSystemPrompt)

{'BE', 'RDNT', 'BETE', 'FXI', 'NFE', 'SNOW', 'GOOGL', 'DELL', 'D', 'SATS', 'BRK.B', 'CRD.B', 'NIO', 'MS', 'ALEX', 'EME', 'COPJ', 'INTT', 'CARR', 'GS', 'SIX', 'AMRX', 'XLU', 'AMZN', 'BA', 'CEG', 'KARS', 'HON', 'EPR', 'BDSX', 'GEO', 'ROK', 'BRK.A', 'PLUG', 'FUN', 'KLIC', 'ALB', 'BCS', 'FUBO', 'AY', 'SQM', 'PNC', 'GOOG', 'ES', 'FIX', 'SCOR', 'ADBE', 'CSIQ', 'FSUGY', 'KOS', 'SIEGY'}
{'https://www.nasdaq.com/articles/fortescue-ltd-depositary-receipt-fsugy-declares-$1.42-dividend', 'https://www.nasdaq.com/articles/dell-stock-weathers-pc-storm-seeking-growth-in-the-ai-revolution', 'https://www.nasdaq.com/articles/adbe-march-2025-options-begin-trading', 'https://www.nasdaq.com/articles/deutsche-bank-downgrades-new-fortress-energy-nfe', 'https://www.nasdaq.com/articles/kosmos-energy-to-offer-$300-mln-of-convertible-senior-notes', 'https://www.nasdaq.com/articles/november-15th-options-now-available-for-geo-group', 'https://www.nasdaq.com/articles/interesting-fxi-put-and-call-options-for-june-202

In [125]:
print(finalAnswer)
print(stocks_string)
print(links_string)


Based on the information gathered, there are several investment opportunities available for companies involved in climate change. One option is investing in ETFs specializing in clean energy or environmental sustainability for exposure to companies leading the transition towards a low-carbon economy, as mentioned in the "Seizing Future Opportunities: BlackRock’s Mega Forces and Thematic ETF Investing" article. Another option is to invest in companies involved in the production of copper, which is vital to the energy transition, according to the "Capture This Year’s Constructive Copper Outlook" article.

There are also specific companies taking action towards climate change, such as Norsk Hydro's renewable energy unit acquiring an 80% stake in a wind power portfolio, as reported in the "Norsk Hydro unit to buy 80% stake in wind power projects" article. New Fortress Energy Inc. offers fully integrated, turnkey energy solutions to accelerate the world's transition to clean energy, as repo